In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Activation,Flatten
from keras.models import Sequential
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-crime/train.csv.zip
/kaggle/input/sf-crime/sampleSubmission.csv.zip
/kaggle/input/sf-crime/test.csv.zip


In [2]:
train_data = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')

In [3]:
train_data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
train_data.shape

(878049, 9)

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


In [6]:
train_data.Category.unique()

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING',
       'SEX OFFENSES NON FORCIBLE', 'EXTORTION', 'GAMBLING', 'BAD CHECKS',
       'TREA', 'RECOVERED VEHICLE', 'PORNOGRAPHY/OBSCENE MAT'],
      dtype=object)

In [7]:
len(train_data.Category.unique().tolist())

39

In [8]:
train_data['Year'] = train_data['Dates'].apply(lambda date:date.split('-')[0]).astype('float')
train_data['Day'] = train_data['Dates'].apply(lambda date:date.split('-')[2].split(' ')[0]).astype('float')
train_data['Month'] = train_data['Dates'].apply(lambda date:date.split('-')[1]).astype('float')

In [9]:
train_data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Year,Day,Month
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015.0,13.0,5.0
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015.0,13.0,5.0
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015.0,13.0,5.0
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015.0,13.0,5.0
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015.0,13.0,5.0


In [10]:
#we can drop resolution,dates column beacause this will not help us to classify crime correctly and will drop the dates column also hence we already extracted date,year,month in separete column
train = train_data.drop(columns = ['Resolution','Dates'],axis = 1)

In [11]:
X = train.loc[:, train.columns != 'Category']
y = train.loc[:, train.columns == 'Category']

In [12]:
y = pd.get_dummies(y['Category'])

In [13]:
X = pd.concat([X,pd.get_dummies(X['PdDistrict'])],axis = 1)
X = pd.concat([X,pd.get_dummies(X['DayOfWeek'])],axis = 1)
X = X.drop(columns = ['Descript','DayOfWeek','PdDistrict','Address'])

In [14]:
X.head()

,X,Y,Year,Day,Month,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,...,SOUTHERN,TARAVAL,TENDERLOIN,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,-122.425892,37.774599,2015.0,13.0,5.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,-122.425892,37.774599,2015.0,13.0,5.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,-122.424363,37.800414,2015.0,13.0,5.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,-122.426995,37.800873,2015.0,13.0,5.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,-122.438738,37.771541,2015.0,13.0,5.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33, random_state=42)

In [17]:
def classification_model():
    model = Sequential()
    model.add(Dense(100,input_shape = (X_train.shape[1],),activation = 'relu'))
    model.add(Dense(80,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(64,activation='relu'))
    #output layes
    model.add(Dense(y_train.shape[1],activation='softmax'))
    
    model.summary()
    return model

In [18]:
model = classification_model()
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2300      
_________________________________________________________________
dense_1 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                5184      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 39)                2

In [19]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588292 entries, 617510 to 121958
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   X           588292 non-null  float64
 1   Y           588292 non-null  float64
 2   Year        588292 non-null  float64
 3   Day         588292 non-null  float64
 4   Month       588292 non-null  float64
 5   BAYVIEW     588292 non-null  uint8  
 6   CENTRAL     588292 non-null  uint8  
 7   INGLESIDE   588292 non-null  uint8  
 8   MISSION     588292 non-null  uint8  
 9   NORTHERN    588292 non-null  uint8  
 10  PARK        588292 non-null  uint8  
 11  RICHMOND    588292 non-null  uint8  
 12  SOUTHERN    588292 non-null  uint8  
 13  TARAVAL     588292 non-null  uint8  
 14  TENDERLOIN  588292 non-null  uint8  
 15  Friday      588292 non-null  uint8  
 16  Monday      588292 non-null  uint8  
 17  Saturday    588292 non-null  uint8  
 18  Sunday      588292 non-null  uint8  
 1

In [20]:
train=model.fit(X_train,y_train, 
         batch_size=64,
         epochs=16,
         verbose=2,
         validation_data=(X_test,y_test))

Epoch 1/16
9193/9193 - 20s - loss: 2.7467 - accuracy: 0.1962 - val_loss: 2.6819 - val_accuracy: 0.1996
Epoch 2/16
9193/9193 - 19s - loss: 2.6840 - accuracy: 0.1989 - val_loss: 2.6829 - val_accuracy: 0.1996
Epoch 3/16
9193/9193 - 19s - loss: 2.6717 - accuracy: 0.2055 - val_loss: 2.6602 - val_accuracy: 0.2120
Epoch 4/16
9193/9193 - 18s - loss: 2.6463 - accuracy: 0.2154 - val_loss: 2.6325 - val_accuracy: 0.2201
Epoch 5/16
9193/9193 - 19s - loss: 2.6366 - accuracy: 0.2172 - val_loss: 2.6259 - val_accuracy: 0.2208
Epoch 6/16
9193/9193 - 19s - loss: 2.6320 - accuracy: 0.2174 - val_loss: 2.6298 - val_accuracy: 0.2175
Epoch 7/16
9193/9193 - 19s - loss: 2.6293 - accuracy: 0.2176 - val_loss: 2.6344 - val_accuracy: 0.2136
Epoch 8/16
9193/9193 - 18s - loss: 2.6275 - accuracy: 0.2182 - val_loss: 2.6264 - val_accuracy: 0.2208
Epoch 9/16
9193/9193 - 19s - loss: 2.6281 - accuracy: 0.2180 - val_loss: 2.6246 - val_accuracy: 0.2207
Epoch 10/16
9193/9193 - 19s - loss: 2.6264 - accuracy: 0.2186 - val_loss:

In [ ]:

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5) 
